In [1]:
from google.colab import files
import requests
import base64
import os

In [ ]:
IMGBB_API_KEY = ""
SERP_API_KEY = ""


In [3]:


uploaded = files.upload()
local_image = list(uploaded.keys())[0]


def upload_to_imgbb(image_path):
    with open(image_path, "rb") as f:
        img_data = base64.b64encode(f.read()).decode('utf-8')
    payload = {
        "key": IMGBB_API_KEY,
        "image": img_data
    }
    response = requests.post("https://api.imgbb.com/1/upload", data=payload)
    if response.status_code == 200:
        return response.json()['data']['image']['url']
    else:
        print(" Upload failed:", response.text)
        return None

def reverse_search_with_serpapi(image_url):
    params = {
        "engine": "google_reverse_image",
        "image_url": image_url,
        "api_key": SERP_API_KEY
    }
    response = requests.get("https://serpapi.com/search", params=params)
    if response.status_code == 200:
        data = response.json()
        return data.get("image_results") or data.get("inline_images") or data.get("shopping_results") or []
    else:
        print(" SerpAPI error:", response.text)
        return []

def display_results(results, keywords):
    print("\n Most Common Product Tags:")
    for k, v in keywords:
        print(f"• {k} ({v} mentions)")

    print("\n🖼️ Visually Similar Results:")
    for r in results[:5]:
        print(f"- Title: {r.get('title')}")
        print(f"  Link: {r.get('link')}")
        print(f"  Thumbnail: {r.get('thumbnail')}")
        print("-----")


img_url = upload_to_imgbb(local_image)
if img_url:
    print(" Image uploaded to:", img_url)
    results = reverse_search_with_serpapi(img_url)

    print("\n Top Visual Matches:")
    for i, r in enumerate(results[:5]):
        print(f"{i+1}. {r.get('title') or r.get('name')}")
        print("   Link:", r.get('link') or 'N/A')
        print("   Thumbnail:", r.get('thumbnail') or r.get('image'))
else:
    print(" Could not upload image.")


Saving a776ff610454.jpg to a776ff610454 (1).jpg
 Image uploaded to: https://i.ibb.co/nNywTfDp/bb7aea46ddb1.jpg

 Top Visual Matches:
1. Daikin 1.5 Ton 5 Star Inverter Split AC (Copper, PM 2.5 Filter ...
   Link: https://www.amazon.in/Daikin-Inverter-Copper-Filter-MTKM50U/dp/B09R4RYCJ4
   Thumbnail: None
2. Whole Home Air Conditioners | Efficient Cooling - Daikin
   Link: https://daikincomfort.com/products/heating-cooling/whole-home-systems/air-conditioners
   Thumbnail: None
3. Daikin 1 Ton 5 Star Inverter Split AC (Copper, PM 2.5 Filter ...
   Link: https://www.amazon.in/Daikin-Inverter-Copper-Filter-MTKM35U/dp/B09R4RXLT4
   Thumbnail: https://serpapi.com/searches/687508e13d3656f0e3f6e122/images/5a0fe3ca3e2b9caaf35369e501ebebe4f777d65c0969be73dbc39abe915b515a.jpeg
4. Daikin 1.5 Ton 3 Star Inverter Split AC (Copper, PM 2.5 Filter ...
   Link: https://www.amazon.in/Daikin-Inverter-Display-Technology-MTKL50U/dp/B0BK1KS6ZD
   Thumbnail: https://serpapi.com/searches/687508e13d3656f0e3f6e12

**GEMINI INTEGRATION**

In [4]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 22.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.12.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have n

In [4]:
from google.colab import files
import requests
import base64
import json
from collections import Counter
import re
import textwrap
import requests
import base64
import json
import re
from collections import Counter
from google.colab import files
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt



In [7]:
GEMINI_URL = f'https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent={GEMINI_API_KEY}'


In [ ]:
# 🔑 Your API Keys
IMGBB_API_KEY = ""
SERP_API_KEY = ""
GEMINI_API_KEY = ""


In [8]:

uploaded = files.upload()
local_image = list(uploaded.keys())[0]

# ✅ Upload to imgbb
def upload_to_imgbb(image_path):
    with open(image_path, "rb") as f:
        img_data = base64.b64encode(f.read()).decode('utf-8')
    payload = {"key": IMGBB_API_KEY, "image": img_data}
    response = requests.post("https://api.imgbb.com/1/upload", data=payload)
    if response.status_code == 200:
        return response.json()['data']['image']['url']
    else:
        print("❌ Upload failed:", response.text)
        return None

# 🔍 Reverse Image Search (SerpAPI)
def reverse_search_with_serpapi(image_url):
    params = {
        "engine": "google_reverse_image",
        "image_url": image_url,
        "api_key": SERP_API_KEY
    }
    response = requests.get("https://serpapi.com/search", params=params)
    if response.status_code == 200:
        data = response.json()
        return data.get("image_results") or data.get("inline_images") or data.get("shopping_results") or []
    else:
        print("❌ SerpAPI error:", response.text)
        return []

# 🧠 Keyword extraction
def extract_keywords_from_titles(titles):
    all_words = []
    for title in titles:
        words = re.findall(r'\b[a-zA-Z0-9]+\b', title.lower())
        keywords = [w for w in words if len(w) > 2 and w not in ['the', 'and', 'for', 'with', 'from', 'this', 'that']]
        all_words.extend(keywords)
    counter = Counter(all_words)
    return counter.most_common(10)

# 🤖 Gemini Vision Analysis
def extract_attributes_with_gemini(image_path):
    with open(image_path, "rb") as image_file:
        image_bytes = image_file.read()
        encoded_image = base64.b64encode(image_bytes).decode("utf-8")

    url = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent"
    headers = {"Content-Type": "application/json"}

    prompt = (
        "You're a professional fashion stylist and image analyst.\n"
        "Given this photo, do the following:\n"
        "1. List fashion objects & accessories (e.g. kurta, jhumka, etc).\n"
        "2. Estimate personal features: skin tone, face shape, makeup, hair.\n"
        "3. Estimate shoulder width or attire size if visible.\n"
        "4. Generate JSON with: preferred_colors, ideal_outfits, accessories, styling_tips.\n"
        "5. Write a short paragraph describing the fashion and styling in detail."
    )

    payload = {
        "contents": [
            {
                "parts": [
                    {
                        "inline_data": {
                            "mime_type": "image/jpeg",
                            "data": encoded_image
                        }
                    },
                    {
                        "text": prompt
                    }
                ]
            }
        ]
    }

    response = requests.post(f"{url}?key={GEMINI_API_KEY}", headers=headers, data=json.dumps(payload))
    if response.status_code == 200:
        return response.json()['candidates'][0]['content']['parts'][0]['text']
    else:
        return f"❌ Gemini API Error:\n{response.text}"

# 📏 MediaPipe Shoulder Width Estimation
def estimate_shoulder_width_cm(image_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=True)
    results = pose.process(image_rgb)

    if not results.pose_landmarks:
        return "❌ No person detected for shoulder width."

    landmarks = results.pose_landmarks.landmark
    h, w, _ = image.shape
    lx = int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w)
    ly = int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h)
    rx = int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w)
    ry = int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h)

    # Distance in pixels
    shoulder_width_px = ((lx - rx) ** 2 + (ly - ry) ** 2) ** 0.5

    # Use eye distance to get px/cm
    eye_l = landmarks[mp_pose.PoseLandmark.LEFT_EYE]
    eye_r = landmarks[mp_pose.PoseLandmark.RIGHT_EYE]
    eye_dist_px = ((eye_l.x * w - eye_r.x * w) ** 2 + (eye_l.y * h - eye_r.y * h) ** 2) ** 0.5
    px_per_cm = eye_dist_px / 16  # avg face width = 16 cm
    shoulder_width_cm = shoulder_width_px / px_per_cm
    return f"{shoulder_width_cm:.2f} cm"

# 🔁 Run Pipeline
img_url = upload_to_imgbb(local_image)
if img_url:
    print("✅ Uploaded to:", img_url)
    results = reverse_search_with_serpapi(img_url)
    titles = [r.get("title") or r.get("name") for r in results if r.get("title") or r.get("name")]

    print("\n🔍 Top Visual Matches:")
    for i, r in enumerate(results[:5]):
        print(f"{i+1}. {titles[i]}")
        print("   Link:", r.get("link") or 'N/A')

    print("\n🧠 Common Keywords:")
    for k, v in extract_keywords_from_titles(titles):
        print(f"• {k} ({v} times)")

    print("\n📏 Estimated Shoulder Width:")
    print(estimate_shoulder_width_cm(local_image))

    print("\n🤖 Gemini Vision Output:")
    result = extract_attributes_with_gemini(local_image)
    print("\n".join(result.splitlines()))
else:
    print("❌ Image upload failed.")


Saving 11850add01ce.jpg to 11850add01ce.jpg
✅ Uploaded to: https://i.ibb.co/Kj8s7znr/8102aad6affb.jpg

🔍 Top Visual Matches:
1. SAINT LAURENT - MEN'S SUMMER 26 SHOW - YouTube
   Link: https://www.youtube.com/watch?v=W-lPWow1lz8
2. Saint Laurent Spring 2025 Ready-to-Wear Collection - WWD
   Link: https://wwd.com/fashion-news/shows-reviews/gallery/saint-laurent-spring-2025-ready-to-wear-collection-1236644634/
3. Fashion Week In Saint Laurent Ysl Runway Best Sale
   Link: https://mychaicafe.com/?v=49033540091680&mod=40c0d2af&uri=shared.php%3Fid%3D1641677-320%26name%3Dfashion+week+in+saint+laurent
4. Saint Laurent Bows Out Of Paris Fashion Week - Grazia
   Link: https://graziamagazine.com/articles/saint-laurent-bows-out-of-paris-fashion-week/
5. Ysl Runway Show 2025 - turizmo.eu
   Link: https://turizmo.eu/story/title/t?t=tur5-36-3727903-3-26-61-ysl+runway+show

🧠 Common Keywords:
• saint (6 times)
• laurent (6 times)
• show (4 times)
• 2025 (3 times)
• fashion (3 times)
• week (2 times)
•

# MergeD with QR Scanner

In [10]:
!pip -q install pillow pyzbar requests pytesseract

In [11]:
!apt-get update && apt-get install -y zbar-tools

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,104 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,266 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2

In [15]:
import requests, base64, json, re, cv2, io, textwrap
from collections import Counter
from PIL import Image
from pyzbar.pyzbar import decode
import mediapipe as mp
import pytesseract
from google.colab import files

In [16]:
def upload_to_imgbb(image_path):
    with open(image_path, "rb") as f:
        img_data = base64.b64encode(f.read()).decode('utf-8')
    r = requests.post("https://api.imgbb.com/1/upload",
                      data={"key": IMGBB_API_KEY, "image": img_data})
    return r.json()["data"]["image"]["url"] if r.ok else None

def qr_lookup(image_path):
    out = []
    codes = decode(Image.open(image_path))
    if not codes:
        return "❌ No QR / barcode detected."
    raw = codes[0].data.decode()
    sym = codes[0].type
    out.append(f"🔎 Detected {sym}: {raw}")

    # extract GTIN
    m = re.search(r"\(01\)(\d{13,14})", raw)
    gtin = m.group(1) if m else re.sub(r"\D", "", raw)
    if not gtin:
        out.append("❌ No numeric GTIN found.")
        return "\n".join(out)
    out.append(f"🔑 GTIN → {gtin}")

    # Open Food Facts
    off = requests.get(
        f"https://world.openfoodfacts.org/api/v0/product/{gtin}.json", timeout=10
    ).json()
    if off.get("status") == 1:
        p = off["product"]
        out.append("🍏 OFF hit ✅ " + (p.get("product_name") or ""))
    else:
        out.append("🍏 OFF: product not found")

    # UPCItemDB
    r = requests.get(
        f"https://api.upcitemdb.com/prod/trial/lookup?upc={gtin}", timeout=10
    )
    if r.status_code == 200 and r.json().get("code") == "OK":
        item = r.json()["items"][0]
        out.append("🛒 UPCItemDB hit ✅ " + (item.get("title") or ""))
    else:
        out.append(f"🛒 UPCItemDB: {r.json().get('message','no record')}")
    return "\n".join(out)

def shoulder_width_cm(path):
    img = cv2.imread(path)
    res = mp.solutions.pose.Pose(static_image_mode=True).process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    if not res.pose_landmarks: return "❌ Shoulder landmarks not found."
    lm, h, w = res.pose_landmarks.landmark, *img.shape[:2][::-1]
    ls, rs = lm[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER], lm[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER]
    px = ((ls.x-lm[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER].x)*w)**2+((ls.y-rs.y)*h)**2
    px = px**0.5
    eye_px = ((lm[mp.solutions.pose.PoseLandmark.LEFT_EYE].x-lm[mp.solutions.pose.PoseLandmark.RIGHT_EYE].x)*w)**2
    eye_px = (eye_px**0.5) or 1
    cm = px/(eye_px/16)
    return f"{cm:.1f} cm"

def gemini_fashion(path):
    enc = base64.b64encode(open(path,"rb").read()).decode()
    prompt = ("You're a professional fashion stylist. 1) List visible fashion items. "
              "2) Note skin tone, face shape, hair. 3) Estimate size if possible. "
              "4) JSON with preferred_colors, ideal_outfits, accessories, styling_tips. "
              "5) Short descriptive paragraph.")
    pl = {"contents":[{"parts":[
          {"inline_data":{"mime_type":"image/jpeg","data":enc}},
          {"text":prompt}
    ]}]}
    r = requests.post(
        f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?key={GEMINI_API_KEY}",
        headers={"Content-Type":"application/json"}, data=json.dumps(pl)
    )
    return r.json()["candidates"][0]["content"]["parts"][0]["text"] if r.ok else "Gemini error"

def serp_keywords(image_url, max_kw=6):
    r = requests.get("https://serpapi.com/search",
                     params={"engine":"google_reverse_image","image_url":image_url,
                             "api_key":SERP_API_KEY})
    if not r.ok: return []
    titles = [x.get("title") or x.get("name") for x in
              (r.json().get("image_results") or r.json().get("inline_images") or [])]
    words = Counter(
        w.lower() for t in titles
        for w in re.findall(r"\b[a-zA-Z]{3,}\b", t)
        if w.lower() not in {'the','and','for','with','from','this','that'}
    )
    return [f"{k} ({v})" for k,v in words.most_common(max_kw)]

# ================================================================
# 3⃣  Upload an image
# ================================================================
up = files.upload()
img_path = next(iter(up))

# ================================================================
# 4⃣  Run full pipeline
# ================================================================
print("\n🖼️  Uploaded image:", img_path)

# 4A barcode / product lookup
print("\n=== QR / Barcode Lookup ===")
print(qr_lookup(img_path))

# 4B shoulder width
print("\n=== Shoulder Width ===")
print(shoulder_width_cm(img_path))

# 4C Gemini Vision
print("\n=== Gemini Fashion Analysis (truncated) ===")
gem = gemini_fashion(img_path)
print(textwrap.shorten(gem, 600, placeholder=" ..."))

# 4D reverse image search keywords
url = upload_to_imgbb(img_path)
if url:
    print("\n=== Reverse‑Image Keywords ===")
    print(", ".join(serp_keywords(url)))
else:
    print("\n(Image upload to imgbb failed — skipping SerpAPI step)")


Saving WhatsApp Image 2025-07-13 at 12.47.01_7b3191ea.jpg to WhatsApp Image 2025-07-13 at 12.47.01_7b3191ea (2).jpg

🖼️  Uploaded image: WhatsApp Image 2025-07-13 at 12.47.01_7b3191ea (2).jpg

=== QR / Barcode Lookup ===
🔎 Detected EAN13: 8901450003394
🔑 GTIN → 8901450003394
🍏 OFF: product not found
🛒 UPCItemDB hit ✅ Denver Hamilton Honour Deodorant Body Spray 165ml/ 5.57oz - Free Shipping

=== Shoulder Width ===
❌ Shoulder landmarks not found.

=== Gemini Fashion Analysis (truncated) ===
I'm designed to process text and images. I'm unable to perform those actions with the provided image. Is there anything else I can do?

=== Reverse‑Image Keywords ===
free (2), custom (2), label (2), printable (1), templates (1), canva (1)


In [17]:
def upload_to_imgbb(image_path):
    with open(image_path, "rb") as f:
        img_data = base64.b64encode(f.read()).decode('utf-8')
    r = requests.post("https://api.imgbb.com/1/upload",
                      data={"key": IMGBB_API_KEY, "image": img_data})
    return r.json()["data"]["image"]["url"] if r.ok else None

def qr_lookup(image_path):
    out = []
    codes = decode(Image.open(image_path))
    if not codes:
        return "❌ No QR / barcode detected."
    raw = codes[0].data.decode()
    sym = codes[0].type
    out.append(f"🔎 Detected {sym}: {raw}")

    # extract GTIN
    m = re.search(r"\(01\)(\d{13,14})", raw)
    gtin = m.group(1) if m else re.sub(r"\D", "", raw)
    if not gtin:
        out.append("❌ No numeric GTIN found.")
        return "\n".join(out)
    out.append(f"🔑 GTIN → {gtin}")

    # Open Food Facts
    off = requests.get(
        f"https://world.openfoodfacts.org/api/v0/product/{gtin}.json", timeout=10
    ).json()
    if off.get("status") == 1:
        p = off["product"]
        out.append("🍏 OFF hit ✅ " + (p.get("product_name") or ""))
    else:
        out.append("🍏 OFF: product not found")

    # UPCItemDB
    r = requests.get(
        f"https://api.upcitemdb.com/prod/trial/lookup?upc={gtin}", timeout=10
    )
    if r.status_code == 200 and r.json().get("code") == "OK":
        item = r.json()["items"][0]
        out.append("🛒 UPCItemDB hit ✅ " + (item.get("title") or ""))
    else:
        out.append(f"🛒 UPCItemDB: {r.json().get('message','no record')}")
    return "\n".join(out)

def shoulder_width_cm(path):
    img = cv2.imread(path)
    res = mp.solutions.pose.Pose(static_image_mode=True).process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    if not res.pose_landmarks: return "❌ Shoulder landmarks not found."
    lm, h, w = res.pose_landmarks.landmark, *img.shape[:2][::-1]
    ls, rs = lm[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER], lm[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER]
    px = ((ls.x-lm[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER].x)*w)**2+((ls.y-rs.y)*h)**2
    px = px**0.5
    eye_px = ((lm[mp.solutions.pose.PoseLandmark.LEFT_EYE].x-lm[mp.solutions.pose.PoseLandmark.RIGHT_EYE].x)*w)**2
    eye_px = (eye_px**0.5) or 1
    cm = px/(eye_px/16)
    return f"{cm:.1f} cm"

def gemini_fashion(path):
    enc = base64.b64encode(open(path,"rb").read()).decode()
    prompt = ("You're a professional fashion stylist. 1) List visible fashion items. "
              "2) Note skin tone, face shape, hair. 3) Estimate size if possible. "
              "4) JSON with preferred_colors, ideal_outfits, accessories, styling_tips. "
              "5) Short descriptive paragraph.")
    pl = {"contents":[{"parts":[
          {"inline_data":{"mime_type":"image/jpeg","data":enc}},
          {"text":prompt}
    ]}]}
    r = requests.post(
        f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?key={GEMINI_API_KEY}",
        headers={"Content-Type":"application/json"}, data=json.dumps(pl)
    )
    return r.json()["candidates"][0]["content"]["parts"][0]["text"] if r.ok else "Gemini error"

def serp_keywords(image_url, max_kw=6):
    r = requests.get("https://serpapi.com/search",
                     params={"engine":"google_reverse_image","image_url":image_url,
                             "api_key":SERP_API_KEY})
    if not r.ok: return []
    titles = [x.get("title") or x.get("name") for x in
              (r.json().get("image_results") or r.json().get("inline_images") or [])]
    words = Counter(
        w.lower() for t in titles
        for w in re.findall(r"\b[a-zA-Z]{3,}\b", t)
        if w.lower() not in {'the','and','for','with','from','this','that'}
    )
    return [f"{k} ({v})" for k,v in words.most_common(max_kw)]

up = files.upload()
img_path = next(iter(up))

print("\n🖼️  Uploaded image:", img_path)

print("\n=== QR / Barcode Lookup ===")
print(qr_lookup(img_path))

print("\n=== Shoulder Width ===")
print(shoulder_width_cm(img_path))

print("\n=== Gemini Fashion Analysis (truncated) ===")
gem = gemini_fashion(img_path)
print(textwrap.shorten(gem, 600, placeholder=" ..."))

url = upload_to_imgbb(img_path)
if url:
    print("\n=== Reverse‑Image Keywords ===")
    print(", ".join(serp_keywords(url)))
else:
    print("\n(Image upload to imgbb failed — skipping SerpAPI step)")


Saving 11850add01ce.jpg to 11850add01ce (1).jpg

🖼️  Uploaded image: 11850add01ce (1).jpg

=== QR / Barcode Lookup ===
❌ No QR / barcode detected.

=== Shoulder Width ===
91.9 cm

=== Gemini Fashion Analysis (truncated) ===
Okay, here's a breakdown of the image based on your request: **1. Fashion Items:** * **Outerwear:** Double-breasted, sequined blazer with a black satin shawl collar. * **Bottom:** Sequined trousers with a satin stripe down the side. * **Shoes:** Black pointed-toe mules or heels with a metallic gold design on the heel. * **Jewellery:** Thin gold necklace with a small pendant. **2. Model Appearance:** * **Skin Tone:** Warm, deep brown/tan. * **Face Shape:** Oval/Heart. * **Hair:** Long, straight, and black, parted down the middle. **3. Size Estimate:** * The model appears to be tall and ...

=== Reverse‑Image Keywords ===
saint (6), laurent (6), show (4), fashion (3), week (2), ysl (2)


SIZE

In [ ]:
# Full implementation of Clothing Size Estimator with Gemini API and MediaPipe (with file upload)

import cv2
import mediapipe as mp
import requests
import base64
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sentence_transformers import SentenceTransformer
from google.colab import files

# MediaPipe setup
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)


# Update Gemini URL to your specified model
GEMINI_URL = (
  "https://generativelanguage.googleapis.com/"
  "v1beta/models/gemini-2.0-flash:generateContent"
  f"?key={GEMINI_API_KEY}"
)


# Text embedding model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Step 1: Pose estimation using MediaPipe
def extract_pose_keypoints(image_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if not results.pose_landmarks:
        raise ValueError("No pose landmarks detected")

    landmarks = results.pose_landmarks.landmark
    keypoints = [(lm.x, lm.y, lm.z) for lm in landmarks]
    return np.array(keypoints).flatten()

# Step 2: Gemini API for image interpretation
def gemini_image_analysis(image_path):
    with open(image_path, 'rb') as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode('utf-8')

    payload = {
        "contents": [{
            "parts": [
                {"text": "Identify clothing item, visible body landmarks (shoulders, chest, waist, hips), and estimate approximate measurements in centimeters."},
                {"inline_data": {"mime_type": "image/jpeg", "data": encoded_image}}
            ]
        }]
    }

    response = requests.post(GEMINI_URL, json=payload)
    if response.status_code == 200:
        description = response.json()['candidates'][0]['content']['parts'][0]['text']
        return description
    else:
        raise Exception(f"Gemini API error: {response.status_code}, {response.text}")

# Step 3: Embedding Gemini description
def embed_description(text):
    return embedder.encode([text])[0]

# Step 4: Train or load your regression model (example simplified here)
def train_regression_model(X, y):
    model = RandomForestRegressor(n_estimators=100)
    model.fit(X, y)
    return model

# Main pipeline function
def predict_body_measurements(image_path, regression_model):
    keypoints = extract_pose_keypoints(image_path)
    gemini_output = gemini_image_analysis(image_path)
    gemini_embedding = embed_description(gemini_output)

    combined_features = np.concatenate([keypoints, gemini_embedding])

    # Predict measurements
    predicted_measurements = regression_model.predict([combined_features])[0]

    return {
        "gemini_description": gemini_output,
        "predicted_shoulder_width_cm": predicted_measurements[0],
        "predicted_chest_width_cm": predicted_measurements[1],
        "predicted_waist_width_cm": predicted_measurements[2],
        "predicted_hip_width_cm": predicted_measurements[3]
    }

# Example usage:
if __name__ == '__main__':
    uploaded = files.upload()
    image_path = list(uploaded.keys())[0]

    # Corrected Dummy training data
    X_dummy = np.random.rand(10, 33*3 + 384)  # Corrected dimension
    y_dummy = np.random.uniform(35, 100, size=(10, 4))

    model = train_regression_model(X_dummy, y_dummy)

    # Predicting
    result = predict_body_measurements(image_path, model)

    print("Gemini's Analysis:\n", result['gemini_description'])
    print("Predicted Shoulder Width (cm):", result['predicted_shoulder_width_cm'])
    print("Predicted Chest Width (cm):", result['predicted_chest_width_cm'])
    print("Predicted Waist Width (cm):", result['predicted_waist_width_cm'])
    print("Predicted Hip Width (cm):", result['predicted_hip_width_cm'])


Saving 7.png to 7.png
Gemini's Analysis:
 Here's the breakdown of the image:

*   **Clothing Items:** The man is wearing a dark blue polo shirt and beige shorts.

*   **Visible Body Landmarks:**
    *   Shoulders
    *   Chest
    *   Waist
    *   Hips

*   **Approximate Measurements (Estimates):** Note, these are very rough estimates based on the image, without real-world reference points, and should be treated as such.

    *   **Shoulder Width:** 48 cm
    *   **Chest Circumference:** 98 cm
    *   **Waist Circumference:** 84 cm
    *   **Hips Circumference:** 94 cm
Predicted Shoulder Width (cm): 60.81391999337092
Predicted Chest Width (cm): 72.11998121457586
Predicted Waist Width (cm): 63.601740027060465
Predicted Hip Width (cm): 65.84979203762454
